In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import  train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pymorphy2
import re
import nltk
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
import lightgbm as lgb
import time

In [3]:
def getCSR_MATRIXfromNPMatrix(Matrix): # построение за numpy array
    return csr_matrix(Matrix)

def getCSR_MATRIXfromPythonMatrix(Matrix): # построение из python list
    return csr_matrix(np.array(Matrix))

def mergeCSRMatrixs(*nums): # слияние любого кол-ва матриц подряд
    first = nums[0]
    for i in range(1, len(nums)):
        first = hstack([first, nums[i]])
    return first

def reduce_dimensions(train_f, test_f):
    cnt_dim = 140
    svd = TruncatedSVD(n_components=cnt_dim, random_state=228, algorithm='arpack')
    train_f_pca = svd.fit_transform(train_f)
    test_f_pca = svd.transform(test_f)
    return train_f_pca, test_f_pca

def combine_all(data, train_f, test_f):
    train_data, test_data = data
    train_data = pd.DataFrame(train_data)
    test_data = pd.DataFrame(test_data)
    other_features = ['lot_price', 'pn_lot_anon']
    for feature in other_features:
        train_data[feature] = train_f[feature]
        test_data[feature] = test_f[feature]
    scaler = StandardScaler()
    if (False):
        train_data[['lot_price']] = scaler.fit_transform(train_data[['lot_price']])
        test_data[['lot_price']] = scaler.transform(test_data[['lot_price']])
    return train_data, test_data

def preprocessing_after_split(train_f, test_f, text_columns=['purchase_name', 'item_descriptions'], cat_columns=['fz', 'region_code', 'okpd2_code']):
    train_f = train_f.fillna(0)
    train_f = train_f[train_f.item_descriptions != 0]
    test_f = test_f.fillna(0)
    test_f = test_f[test_f.item_descriptions != 0]
    train_f = train_f.reset_index()
    test_f = test_f.reset_index()

    # one-hot
    train_f[cat_columns].fillna('None', inplace=True)
    oh_enc = OneHotEncoder(handle_unknown='ignore')
    train_f_oh = oh_enc.fit_transform(train_f[cat_columns])
    test_f_oh = oh_enc.transform(test_f[cat_columns])
    train_dataCSR = train_f_oh
    test_dataCSR = test_f_oh
    #tf-idf
    if (False):
        for name in text_columns:
            vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=666, decode_error=
                                    'replace')
            train_f_tfidf = vectorizer.fit_transform(train_f[name])
            test_f_tfidf = vectorizer.transform(test_f[name])
            # train_dataCSR = mergeCSRMatrixs(train_dataCSR, train_f_tfidf)
            # test_dataCSR = mergeCSRMatrixs(test_dataCSR, test_f_tfidf)
            print(train_dataCSR.shape)
        # print(vectorizer.get_feature_names())
    if (True):
        train_f['mean_region_price'] = np.full(len(train_f), 0)
        test_f['mean_region_price'] = np.full(len(test_f), 0)
        regions = set(train_f['region_code'].values)
        for region in regions: 
            prices = train_f[train_f['region_code'] == region]['lot_price']
            prices = prices.apply(np.exp)
            now_mean = np.mean(prices)
            now_mean = np.log(now_mean)
            train_f.loc[train_f['region_code'] == region, 'mean_region_price'] = now_mean
            test_f.loc[test_f['region_code'] == region, 'mean_region_price'] = now_mean
        if (False): # Worse
            train_f['mean_region_price'] -= train_f['mean_region_price'].min()
            train_f['mean_region_price'] /= train_f['mean_region_price'].max()
            test_f['mean_region_price'] -= test_f['mean_region_price'].min()
            test_f['mean_region_price'] /= test_f['mean_region_price'].max()
        if (True):
            train_dataCSR = mergeCSRMatrixs(train_dataCSR, getCSR_MATRIXfromNPMatrix(train_f['mean_region_price'].values.reshape(len(train_f), 1)))
            test_dataCSR = mergeCSRMatrixs(test_dataCSR, getCSR_MATRIXfromNPMatrix(test_f['mean_region_price'].values.reshape(len(test_f), 1)))
    if (True):
        train_f['mean_okpd_price'] = np.full(len(train_f), 0)
        test_f['mean_okpd_price'] = np.full(len(test_f), 0)
        okpds = set(train_f['okpd2_code'].values)
        for okpd in okpds: 
            prices = train_f[train_f['okpd2_code'] == okpd]['lot_price']
            prices = prices.apply(np.exp)
            now_mean = np.mean(prices)
            now_mean = np.log(now_mean)
            train_f.loc[train_f['okpd2_code'] == okpd, 'mean_okpd_price'] = now_mean
            test_f.loc[test_f['okpd2_code'] == okpd, 'mean_okpd_price'] = now_mean
        if (False): # Worse
            train_f['mean_okpd_price'] -= train_f['mean_okpd_price'].min()
            train_f['mean_okpd_price'] /= train_f['mean_okpd_price'].max()
            test_f['mean_okpd_price'] -= test_f['mean_okpd_price'].min()
            test_f['mean_okpd_price'] /= test_f['mean_okpd_price'].max()
        if (True):
            train_dataCSR = mergeCSRMatrixs(train_dataCSR, getCSR_MATRIXfromNPMatrix(train_f['mean_okpd_price'].values.reshape(len(train_f), 1)))
            test_dataCSR = mergeCSRMatrixs(test_dataCSR, getCSR_MATRIXfromNPMatrix(test_f['mean_okpd_price'].values.reshape(len(test_f), 1)))

    return combine_all(reduce_dimensions(train_dataCSR, test_dataCSR), train_f, test_f)

In [4]:
train = pd.read_csv("C:\\Users\\Timofey\\Downloads\\data\\train_data_lem_transformed.csv", sep=';')
test = pd.read_csv("C:\\Users\\Timofey\\Downloads\\data\\test_data_lem_transformed.csv", sep=';')
logs = pd.read_csv("C:\\Users\\Timofey\\Downloads\\data\\train_labels.csv", sep=';')

In [5]:
train, test = preprocessing_after_split(train, test)
X_train = train.loc[:, train.columns[:-1]]
X_test = test.loc[:, test.columns[:-1]]
participants = list(set(logs['participant_inn_kpp_anon']))
X_train.head(2)

c:\users\timofey\anaconda3\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,lot_price
0,21.273966,-0.168336,-0.236909,-0.096481,-0.052230,-0.056621,-0.140249,-0.053548,-0.290052,0.697916,...,0.007405,0.007752,0.013068,-0.014440,-0.005070,0.025585,0.009646,-0.012025,-0.030783,11.723996
1,20.840299,0.271440,-0.234627,-0.138281,-0.033076,-0.060267,-0.168655,-0.050029,-0.305296,0.693035,...,-0.046984,-0.023759,-0.058013,-0.018313,-0.004027,-0.034842,-0.009740,-0.013941,0.028972,11.877569


In [19]:
def recommendation_system(X_train, X_test, train, test, participants, left_bound, right_bound):
    start = time.time()
    cnt_good = 0
    cnt_total = 0
    train_pn = train['pn_lot_anon'].values
    test_pn = test['pn_lot_anon'].values
    res = pd.DataFrame(columns=['inn_kpp', 'actual_recommended_pn_lot', 'similarity_score'])
    for p_id in range(left_bound, right_bound + 1):
        if p_id % 10 == 0:
            print(p_id)
        y_train_ = np.full(len(X_train), 0)
        current_p = participants[p_id]
        parts_inn = logs[logs['participant_inn_kpp_anon'] == current_p]
        p_numb = list(set(parts_inn['pn_lot_anon']))
        for i in range(len(y_train_)):
            if train_pn[i] in p_numb:
                y_train_[i] = 1
                if sum(parts_inn[logs['pn_lot_anon'] == train_pn[i]]['is_winner']) == 1:
                    y_train_[i] = 2
        if(sum(y_train_) == 0):
            continue
        model = lgb.LGBMRegressor(n_estimators=77, max_depth=7, num_leaves=83)
        model.fit(X_train, y_train_)
        y_pred = model.predict(X_test)
        scores = pd.DataFrame(columns=['pred', 'lot'])
        scores['pred'] = y_pred
        scores['lot'] = test['pn_lot_anon']
        scores = scores.sort_values(by='pred', ascending=False)
        for i in range(35):
            res = res.append({'inn_kpp' : current_p, 'actual_recommended_pn_lot' : scores.loc[scores.index[i], 'lot'], 'similarity_score' : scores.loc[scores.index[i], 'pred']}, ignore_index=True)
    print(time.time() - start)
    return res

In [7]:
result = recommendation_system(X_train=X_train, X_test=X_test, train=train, test=test, participants=participants, left_bound=0, right_bound=2999)

0


<ipython-input-6-4f6d7c19796a>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sum(parts_inn[logs['pn_lot_anon'] == train_pn[i]]['is_winner']) == 1:


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
27088.647139787674


In [8]:
result.to_csv('score3000.csv', sep=';', index=False)

In [9]:
result = recommendation_system(X_train=X_train, X_test=X_test, train=train, test=test, participants=participants, left_bound=3000, right_bound=4499)

3000


<ipython-input-6-4f6d7c19796a>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sum(parts_inn[logs['pn_lot_anon'] == train_pn[i]]['is_winner']) == 1:


3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
13608.086286306381


In [10]:
result.to_csv('score4500.csv', sep=';', index=False)

In [14]:
result = recommendation_system(X_train=X_train, X_test=X_test, train=train, test=test, participants=participants, left_bound=16400, right_bound=len(participants))

16400


<ipython-input-6-4f6d7c19796a>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sum(parts_inn[logs['pn_lot_anon'] == train_pn[i]]['is_winner']) == 1:


IndexError: list index out of range

In [15]:
result

,inn_kpp,actual_recommended_pn_lot,similarity_score
0,inn_kpp_4130158,pn_lot_9694797,0.395861
1,inn_kpp_4130158,pn_lot_2980427,0.369230
2,inn_kpp_4130158,pn_lot_3401680,0.369230
3,inn_kpp_4130158,pn_lot_1182013,0.274128
4,inn_kpp_4130158,pn_lot_573704,0.249243
...,...,...,...
52460,inn_kpp_7863391,pn_lot_4792795,0.067706
52461,inn_kpp_7863391,pn_lot_4300072,0.060721
52462,inn_kpp_7863391,pn_lot_1743838,0.060710
52463,inn_kpp_7863391,pn_lot_8859414,0.057779


In [16]:
result.to_csv('score16500.csv', sep=';', index=False)

In [20]:
result = recommendation_system(X_train=X_train, X_test=X_test, train=train, test=test, participants=participants, left_bound=11500, right_bound=11699)

11500


<ipython-input-19-af010faa406b>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sum(parts_inn[logs['pn_lot_anon'] == train_pn[i]]['is_winner']) == 1:


11510
11520
11530
11540
11550
11560
11570
11580
11590
11600
11610
11620
11630
11640
11650
11660
11670
11680
11690
2014.7503905296326


In [21]:
result.to_csv('score11700.csv', sep=';', index=False)